In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')
import os
drive_path='/content/drive/Mydrive/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


# Define paths
base_dir = '/content/drive/MyDrive/dataset'
csv_path = os.path.join(base_dir, 'test.csv')  # Adjust if your CSV file is named differently
image_dir = base_dir  # Images are inside the subfolders of `dataset`

# Load CSV file
df = pd.read_csv(csv_path)

# Map image paths to full paths
df['image_path'] = df['Image_id'].apply(lambda x: os.path.join(image_dir, x))
df = df[df['image_path'].apply(lambda x: os.path.exists(x))]  # Filter rows where images exist

# Check data
print(f"Number of images in dataset: {len(df)}")
print(df.head())

# Split the dataset into training and validation sets
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['labels'], random_state=42)

# Define parameters for image processing
img_height, img_width = 224, 224
batch_size = 32

# Use ImageDataGenerator to load and augment images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

# Create training and validation generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='labels',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='labels',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Verify the class indices
print("Class Indices:", train_generator.class_indices)

Number of images in dataset: 0
Empty DataFrame
Columns: [Image_id, labels, image_path]
Index: []


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:


# Define the dataset path
base_dir = '/content/drive/MyDrive/dataset'  # Replace with the actual dataset folder path

# Define parameters for image processing
img_height, img_width = 224, 224  # Image size for input
batch_size = 32

# Use ImageDataGenerator to load and augment images
train_datagen = ImageDataGenerator(
    rescale=1.0/255,          # Normalize pixel values to [0, 1]
    rotation_range=30,        # Randomly rotate images
    width_shift_range=0.2,    # Shift images horizontally
    height_shift_range=0.2,   # Shift images vertically
    shear_range=0.2,          # Apply shear transformation
    zoom_range=0.2,           # Apply zoom
    horizontal_flip=True,     # Flip images horizontally
    validation_split=0.2      # Use 20% of the data for validation
)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use training subset
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use validation subset
)

# Verify class indices
print("Class Indices:", train_generator.class_indices)

Found 1224 images belonging to 6 classes.
Found 306 images belonging to 6 classes.
Class Indices: {'alien_test': 0, 'cloudy': 1, 'foggy': 2, 'rainy': 3, 'shine': 4, 'sunrise': 5}


In [ ]:
# Model definition
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 6 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Calculate steps per epoch
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = val_generator.samples // val_generator.batch_size

# Train the model
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)

# Save the model
model.save('weather_classification_model.h5')

Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 199s 5s/step - accuracy: 0.3835 - loss: 1.9960 - val_accuracy: 0.5938 - val_loss: 1.1751
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.4375 - loss: 1.4007 - val_accuracy: 0.5000 - val_loss: 1.1948
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 204s 5s/step - accuracy: 0.5808 - loss: 1.1320 - val_accuracy: 0.5903 - val_loss: 1.1002
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.7812 - loss: 0.8128 - val_accuracy: 0.7222 - val_loss: 1.0157
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 203s 5s/step - accuracy: 0.6732 - loss: 0.9335 - val_accuracy: 0.5729 - val_loss: 1.3135
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.6562 - loss: 1.0239 - val_accuracy: 0.5556 - val_loss: 1.1560
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 242s 5s/step - accuracy: 0.6706 - loss: 0.9236 - val_accuracy: 0.6736 - val_loss: 0.9193
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.5312 - loss: 1.2256 - val_accuracy: 0.7778 - v

In [ ]:
# Save the model
model.save('weather_classification_model.keras')